In [18]:
import chipwhisperer as cw
import time

# Connect to the ChipWhisperer hardware
scope = cw.scope()
target = cw.target(scope)
target.baud = 9600

In [19]:
# GLITCH 
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "enable_only"
# scope.glitch.output = "glitch_only"
scope.glitch.trigger_src = "manual"
scope.glitch.trigger_src = "ext_continuous"
scope.glitch.repeat = 22
scope.glitch.ext_offset = 1  
    
# CLOCK
# scope.clock.clkgen_freq = 32E6 # Overclocking can make glitching better, max freq: 81Mhz
scope.clock.clkgen_freq = 96E6

# IO
scope.io.glitch_lp = True
scope.io.glitch_hp = True
scope.io.tio1 = "high_z" # Read pin from Renesas to see if glitch worked
scope.io.tio2 = "serial_rx" # Can be used to receive serial 
scope.io.tio3 = "gpio_high" # To /RESET pin
scope.io.tio4 = "high_z"
scope.io.hs2 = "clkgen" # Attach to clockinput Renesas chip EXTAL
scope.io.hs2 = "glitch" # Attach to clockinput Renesas chip EXTAL

# TRIGGER
scope.trigger.triggers = "tio4"

print(scope.trigger)

def reset_mcu():
    """Reset the microcontroller by toggling IO3 low."""
    target.flush()
    # print(".", end="")
    scope.io.tio3 = "gpio_low"  # Set IO3 as GPIO to reset the MCU
    time.sleep(0.01)  # Wait for 10ms
    scope.io.tio3 = "gpio_high"  # Set IO3 as GPIO to reset the MCU
    # time.sleep(0.01)  # Wait for 10ms
    


reset_mcu()

# Good settings:
# 32Mhz / 16 repeat / 3.345V
# 50 tries
# successes = 18, repeat = 15
# successes = 10, repeat = 16
# successes = 20, repeat = 17



triggers = tio4
module   = basic



In [38]:
# scope.io.tio1 = "serial_tx"
# scope.io.tio2 = "serial_rx"
# target.baud = 9600


# # Send data (optional)
# target.write("Hello, ChipWhisperer!\n")
# print("start")
reset_mcu()
# time.sleep(2)
# # Receive data
while True:
    # reset_mcu()
    # time.sleep(1)
    if target.in_waiting:
        #print("something available: ", target.in_waiting, target.read(0))
        data = target.read()
        if len(data) > 0:
            print(data)


#         time.sleep(1)

 
S


KeyboardInterrupt: 

In [26]:
# gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc = cw.GlitchController(groups=["success", "reset", "normal", "bigsuccess", "timeout"], parameters=["ext_offset", "repeat", "tries"])
# gc = cw.GlitchController(groups=["success", "reset", "normal", "bigsuccess"], parameters=["width", "offset", "repeat", "tries"])
gc.display_stats()


num_tries = 5 # increase to get better glitch stats
gc.set_range("tries", 1, num_tries)

# gc.set_range("ext_offset", 22, 22)
# gc.set_range("repeat", 1, 8000)

gc.set_range("ext_offset", 1, 200)
gc.set_range("repeat", 1, 200)

# gc.set_range("ext_offset", 53, 150)
# gc.set_range("repeat", 31, 100)

# gc.set_range("ext_offset", 1, 200)
# gc.set_range("repeat", 1, 60)

# gc.set_range("ext_offset", 1, 1)
# gc.set_range("repeat", 1, 1)

# gc.set_range("width", -48, 48)
# gc.set_range("offset", 0, 48)
# gc.set_range("ext_offset", 7, 10)

gc.set_global_step(1)


# gc.set_step("width", 0.5)
# gc.set_step("offset", 1)
# gc.set_step("ext_offset", 1)
gc.set_step("repeat", 1)




IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

IntText(value=0, description='bigsuccess count:', disabled=True)

IntText(value=0, description='timeout count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

FloatSlider(value=0.0, continuous_update=False, description='repeat setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='tries setting:', disabled=True, max=10.0, readout…

In [ ]:
import matplotlib.pyplot as plt
#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

reset_mcu()
scope.arm()

highest_byte_count = -1
best_parameter_value = -1

resultss = []  # To store the success counts for each repeat value
parameter_values = []

def glitch_worked():
    return not scope.io.tio_states[0]


def keep_reading_untill_stops(data, filename):
    full_data = data

    time.sleep(0.01)
    print("start reading")
    while target.in_waiting():
        print('.', end="")
        full_data += target.readbytes()
        time.sleep(0.01)

    print("done reading")

    total_bytes = len(full_data)

        # Save the full data to a binary file
    with open(f"glitch_dumps/{filename}_{str(total_bytes)}.bin", 'wb') as bin_file:
        bin_file.write(full_data)

    return total_bytes


for glitch_setting in gc.glitch_values():
    scope.glitch.ext_offset = glitch_setting[0]
    scope.glitch.repeat = glitch_setting[1]
    try_counter = glitch_setting[2]

    reset_mcu()

    time.sleep(0.1)  # Short delay for to check if it worked

    if target.in_waiting():
        data = target.readbytes()
        if len(data) > 0:

            if b'S' in data:  # Check if the byte for 'S' is in the data
                if b'SVEN' in data:  # Check if the byte sequence for "SVEN" is in the data
                    
                    gc.add("bigsuccess")
                    

                    count = keep_reading_untill_stops(data, filename=f"{str(scope.glitch.repeat)}_{str(scope.glitch.ext_offset)}")
                    parameter_values.append(glitch_setting[1])
                    resultss.append(glitch_setting[0])
                    if count > highest_byte_count:
                        highest_byte_count = count
                        best_parameter_value = glitch_setting[0]

                    print(f"SUCCES: glitch ext offset: {glitch_setting[0]} repeat: {glitch_setting[1]} bytes: {count} | best so far: {highest_byte_count} bytes at setting {best_parameter_value}")

                else:
                    if len(data) > 2:
                        gc.add("success") 
                    else:
                        gc.add("normal")

        
    
    else:
         gc.add("reset")


print("Done glitching")


print("Best", highest_byte_count, num_tries, best_parameter_value)

plt.figure()
plt.plot(parameter_values, resultss, marker='o')
plt.title("ext offset vs Repeat Value")
plt.xlabel("Repeat Value")
plt.ylabel(f"ext_offset")
plt.grid()
plt.show()

#enable logging
cw.set_all_log_levels(cw.logging.WARNING)
gc.plot_2d(plotdots={"success":"+g", "bigsuccess":"+b", "reset":"xr", "normal":None}, x_index="ext_offset", y_index="repeat")

SUCCESS: glitch ext offset: 6 repeat: 13 bytes: 259 | best so far: 259 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 21 bytes: 131 | best so far: 259 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 21 bytes: 259 | best so far: 259 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 23 bytes: 515 | best so far: 515 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 25 bytes: 259 | best so far: 515 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 33 bytes: 515 | best so far: 515 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 33 bytes: 11 | best so far: 515 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 37 bytes: 114 | best so far: 515 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 39 bytes: 515 | best so far: 515 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 55 bytes: 3372 | best so far: 3372 bytes at setting 6
SUCCESS: glitch ext offset: 6 repeat: 55 bytes: 2466 | best so far: 3372 bytes at setting 6
SUCCESS

Timeout: Skipping glitch setting 19, 13
Timeout: Skipping glitch setting 19, 14


Timeout: Skipping glitch setting 19, 14


Timeout: Skipping glitch setting 19, 14


Timeout: Skipping glitch setting 19, 14


...

Timeout: Skipping glitch setting 19, 42


Timeout: Skipping glitch setting 19, 42


Timeout: Skipping glitch setting 19, 42


Timeout: Skipping glitch setting 19, 43


USBErrorPipe: LIBUSB_ERROR_PIPE [-9]

In [10]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})
# gc.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="ext_offset", y_index="repeat")
gc.plot_2d(plotdots={"success":"+g", "bigsuccess":"+b", "reset":"xr", "normal":None}, x_index="ext_offset", y_index="repeat")
# results = gc.calc(sort="success_rate")
# results
# gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, )
# print(gc.results)

:Overlay
   .Points.I          :Points   [x,y]
   .Points.Success    :Points   [ext_offset,repeat]   (success_rate)
   .Points.Reset      :Points   [ext_offset,repeat]   (reset_rate)
   .Points.Bigsuccess :Points   [ext_offset,repeat]   (bigsuccess_rate)

In [190]:
scope.glitch.repeat = 100
scope.glitch.width = 25
scope.clock.clkgen_freq = 96E6
scope.glitch.output = "enable_only"
# scope.glitch.trigger_src = "manual"

for _ in range(1000):
    scope.arm()
    # reset_mcu()
# gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})
# gc.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="repeat", y_index="tries")
# results = gc.calc(ignore_params=["ext_offset"], sort="success_rate")
# results
# gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, )
# print(gc.)

KeyboardInterrupt: 

In [ ]:
scope.dis()
target.dis()